<center><font size =6, color="marquee"> Twitter US Airline Sentiment Classification using 
    navie bayes generative classifier </font></center>

  https://www.kaggle.com/crowdflower/twitter-airline-sentiment
  
  Data can be downloaded from the following link. You have to login to the kaggle site to get access to the data.
Unzip the data in folder where notebook is running. There should be a file called **Tweet.csv**

https://www.kaggle.com/crowdflower/twitter-airline-sentiment/downloads/twitter-airline-sentiment.zip/2

<img src="https://images.unsplash.com/photo-1529990131237-cfa5ce9517b6?ixlib=rb-1.2.1&auto=format&fit=crop&w=1949&q=80"  width="900" height="400">

This excercise will help us to get started with navie bayes generative classifier and using tools(pandas, sklearn etc) from python eco system. Most of the code is already written but please understand it.

**Problem statement:** classify sentiment based on text as positive or negative* using **naive bayes** in supervised machine setting.
See this link to get an idea supervised learning workflow [supervsed learning workflow](http://www.allprogrammingtutorials.com/tutorials/introduction-to-machine-learning.php)

**Dataset:** See the link above


credit:


- some of the images are from  https://unsplash.com


In [1]:
#Must for inline plot
%matplotlib inline 
import numpy as np
import pandas as pd # for data analysis
import matplotlib.pyplot as plt # for plotting



In [2]:
# You may have to install following libraries too. Please uncomment to install
#pip install scikit-learn
!pip install -U gensim
#!pip install -U setuptools
#!conda install pattern
#! install --upgrade pip

Requirement already up-to-date: gensim in /Users/rjdaskevich/opt/anaconda3/lib/python3.8/site-packages (4.0.1)


In [3]:
#!ls
data_url = 'Tweets.csv'
 

In [4]:
!head -n 5  Tweets.csv

tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
570306133677760513,neutral,1.0,,,Virgin America,,cairdin,,0,@VirginAmerica What @dhepburn said.,,2015-02-24 11:35:52 -0800,,Eastern Time (US & Canada)
570301130888122368,positive,0.3486,,0.0,Virgin America,,jnardino,,0,@VirginAmerica plus you've added commercials to the experience... tacky.,,2015-02-24 11:15:59 -0800,,Pacific Time (US & Canada)
570301083672813571,neutral,0.6837,,,Virgin America,,yvonnalynn,,0,@VirginAmerica I didn't today... Must mean I need to take another trip!,,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
570301031407624196,negative,1.0,Bad Flight,0.7033,Virgin America,,jnardino,,0,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",,2015-02-24 11:

Clearly it is a comma separated csv file.


First step before jumping into using any machine learning model is understanding the data by **describing it's statistical attribute and visualizating samples or sample property**.
We can use CSV file reader and try to accomplish above task. But as they say python is a language with **battery(libraries) included**. Let's use **pandas and matplotlib** libraries to do this task as cleanly as possible. What to describe and what to plot will be an essential skill we build as we do various data science or machine learning tasks. Also with time you will also built a knowledge of various packages available for different domain in python eco system. Most of the time reading blog and google search does the job of finding right libraries. Various packages for download and installation are avaiable at [PyPI - the Python Package Index](https://pypi.python.org/pypi)

**Optional**
This is [10 Minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/10min.html)

If you have more time look into this link [Pandas Tutorial: DataFrames in Python](https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python#gs.dEdNuDM)

Let's load the csv file in pandas dataframe

In [5]:
# You will see how wrapping the file in pandas simplify lot of tasks
airlines_sentiment_df = pd.read_csv('Tweets.csv')
airlines_sentiment_df.head(6) # there are other functions like tail and sample to check record in dataset


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
5,570300767074181121,negative,1.0000,Can't Tell,0.6842,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica seriously would pay $30 a fligh...,NaN,2015-02-24 11:14:33 -0800,NaN,Pacific Time (US & Canada)


In [6]:
airlines_sentiment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

# Let's select observation with positive and negative sentiment only.

In [7]:
airlines_sentiment_no_neutral = airlines_sentiment_df[airlines_sentiment_df.airline_sentiment.isin(['positive', 'negative'])]

airlines_sentiment_df = airlines_sentiment_no_neutral

In [8]:
#Let's view some random record
#airlines_sentiment_df.sample(10)
airlines_sentiment_df.count()

tweet_id                        11541
airline_sentiment               11541
airline_sentiment_confidence    11541
negativereason                   9178
negativereason_confidence        9508
airline                         11541
airline_sentiment_gold             37
name                            11541
negativereason_gold                32
retweet_count                   11541
text                            11541
tweet_coord                       838
tweet_created                   11541
tweet_location                   7770
user_timezone                    7692
dtype: int64

In [9]:
airlines_sentiment_no_neutral.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')

In [10]:
airlines_sentiment_no_neutral['airline_sentiment'].value_counts()

negative    9178
positive    2363
Name: airline_sentiment, dtype: int64

Let's select only airline_sentiment and text filed.

In [11]:
airlines_sentiment_no_neutral = airlines_sentiment_no_neutral[['airline_sentiment', 'text']]
print(type(airlines_sentiment_no_neutral))

<class 'pandas.core.frame.DataFrame'>


In [12]:
print(airlines_sentiment_no_neutral.head())
airlines_sentiment_no_neutral.count()

  airline_sentiment                                               text
1          positive  @VirginAmerica plus you've added commercials t...
3          negative  @VirginAmerica it's really aggressive to blast...
4          negative  @VirginAmerica and it's a really big bad thing...
5          negative  @VirginAmerica seriously would pay $30 a fligh...
6          positive  @VirginAmerica yes, nearly every time I fly VX...


airline_sentiment    11541
text                 11541
dtype: int64

**This is our  data set $\mathcal{D} = \{({x_i}, y_i)\}_{i=1}^{N=5574}$ $x_i$ is text field and $y_i$ is airlines_sentiment(positive or negative)**. Using using this we will train(learn parameters $\theta$ of a models(Naive bayes, Discriminant anlaysis based etc.)) and use trained model to classify new text as positive or negative.


In [13]:
airlines_sentiment_no_neutral.dtypes



airline_sentiment    object
text                 object
dtype: object

airline_sentiment attribute is not in proper format but we will use it's string value later to create class label(positive=1, negative =0)

In [14]:
airlines_sentiment_no_neutral.isna().sum()

airline_sentiment    0
text                 0
dtype: int64

Dataset doesn't contain any nan and  looks clean.

Typically we should do an extensive EDA(exploratory data analysis) on this data to see distribution of various attribute on **positive** and **negative** class. This can give some more insight into distriubtion of text/words in these two classes and help in further cleanup of text  data.

We'll go ahead use the current text field and sentiment label to build naive bayes classifier.

## Computer only understand scalar or vector or matrices. We need to convert text to vectors(feature).


# Feature engineering

we'll use the [Bag-of-words model](https://en.wikipedia.org/wiki/Bag-of-words_model) approach for creating feature
representing our  message.

### Bag of words model for document:

In BOW  we treat document as collection of word without any order. 

- **Bernoulli document model:** message is represented by a binary feature vector of *absence or presence of word*.
- **Multinomial document model**: message is represented by an *integer feature vector of word frequency*.

 Later we will see
there are better model for sentence or document representation **where words order matters**. There are model which takes into account the word order like [N-gram](https://en.wikipedia.org/wiki/N-gram) etc.
Infact, Deep learning has enabled us to learn even better embedding of words using context of words(co-occurrence ).
We will try to use them in **deep learning section** [**optional** see [GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/projects/glove/)]

**Let's build Multinomial document model**

Converting to vector is bit involved and require a good understanding of NLP(natural language processing).

But in general, to convert a message into vector we need to:
1. convert a sentence into word token
2. Normalize the words(lemmatization, stemming), removing stop words, Capital form(Cow vs cow) handling etc.
3. Build a dictionary of words and map the messages into vector of counts using this dictionary.
4. Finally train a  Naive Bayes model.

Again can use a python library [nltk](https://www.nltk.org/) to perform above task as per our need(in tools1 we did similar exercises) but current exercise we'll use another common library, **gensim** to do the basic cleanup.


**Let's only do [lemmatization](https://en.wikipedia.org/wiki/Lemmatisation)**

In [15]:
airlines_sentiment_df = airlines_sentiment_no_neutral
airlines_sentiment_df.head()

,airline_sentiment,text
1,positive,@VirginAmerica plus you've added commercials t...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
5,negative,@VirginAmerica seriously would pay $30 a fligh...
6,positive,"@VirginAmerica yes, nearly every time I fly VX..."


In [16]:
!pip install simplemma

In [17]:
import simplemma
#from pattern.en import parse
#from gensim.utils import lemmatize
#gensim.utils.lemmatize

Let' see how it works on an example.

In [18]:
langdata = simplemma.load_data('en')
langdata1 = simplemma.load_data('es')
test = ['Hi', 'Machine' , 'learning','bueno']
print([simplemma.lemmatize(token, langdata, langdata1) for token in test])
print([simplemma.lemmatize(token, langdata, langdata1) for token in test])

test = "short shortest shorter"
print(simplemma.simple_tokenizer(test))

#and both

print(simplemma.text_lemmatizer(test, langdata))


['hello', 'machine', 'learn', 'bueno']
['hello', 'machine', 'learn', 'bueno']
['short', 'shortest', 'shorter']
['short', 'short', 'short']


As you can see apart from lemmatizing we also get part of speech attached to the word

# Q1(.75 points). Finish the following lambda so that we only capture lemmatized words. *apply* will apply  written lambda to each text value.

Hint:
Your preprocessed_messages(a pandas Series) should look like

1    [virginamerica, ve, add, commercial, experienc...

3    [virginamerica, really, aggressive, blast, obn...

In [19]:
# Finish this lambda
preprocessed_messages= airlines_sentiment_df.text.apply(lambda text: [word for word in simplemma.text_lemmatizer(text, langdata)])
type(preprocessed_messages)

pandas.core.series.Series

Let see how lemmatization looks likes.

In [20]:
preprocessed_messages.head()

1    [@VirginAmerica, plus, you, ', have, add, comm...
3    [@VirginAmerica, it, ', s, really, aggressive,...
4    [@VirginAmerica, and, it, ', s, a, really, big...
5    [@VirginAmerica, seriously, would, pay, $30, a...
6    [@VirginAmerica, yes, ,, nearly, every, time, ...
Name: text, dtype: object

As you can see this is very basic clean and further clean can be done like removing more stop words, spell correction etc.

In the first text, see how **added** to changed to **add**. We can do some more pre processing  with some justifucation but let's go
ahead with current manipulation.

There are lots of other natural language processing libraries in python for performing above activities like
- [Spacy](https://spacy.io/)
- [nltk](https://www.nltk.org/)

- One of the fundamental concern in machine learning is *Generalization*, **how well our machine is going to work on unseen/future data. Does it generalize well on future data**?
- If we wanted to do well on given data, why  would we even bother to build an algorithm. We can just store the data and do a lookup for any message.

We will come back to this question later in the course when we talk about **model selection and evaluation**.

One simple way to answer to above question is to hide some portion of dataset and use remaining dataset for building the model i.e. learning the parameters of the model. Once we have build the model, we can report some number/measure on hidden dataset to tell how well the model will perform on unseen data.

In [21]:
messages_labels = airlines_sentiment_df.airline_sentiment
messages_labels.head()


1    positive
3    negative
4    negative
5    negative
6    positive
Name: airline_sentiment, dtype: object

# Let's partition our data into training and test set

In [22]:
from sklearn.model_selection import train_test_split

training_messages, test_messages, training_labels, test_labels = train_test_split(preprocessed_messages,
                                                                                 messages_labels,
                                                                                test_size= .2,
                                                                                 stratify= messages_labels)
print('Shape of training X {} and train Y {}'.format(training_messages.shape, training_labels.shape))
print('Shape of test X {} and test Y {}'.format(test_messages.shape, test_labels.shape))

Shape of training X (9232,) and train Y (9232,)
Shape of test X (2309,) and test Y (2309,)


**We'll use training messages only for building the model**

- We need to convert each message into count vector. Where a positive/negative message is mapped to vector representing each word frequency in the message
    + To do this we need to build a dictionary of words first

# Q 2(.25 point) Write code to update the python set. We are using set as it takes care of duplicate words.

In [23]:
unique_word = set()
print(type(unique_word))

for message in training_messages:
    unique_word.update(message)
    
unique_word

<class 'set'>


{'bad',
 'tmrw',
 'mistreat',
 'CAE',
 'former',
 'nbr',
 'dream',
 'myself',
 'dunno',
 '8273993',
 'Airserv',
 'functionally',
 'Gonna',
 '#luvforSW',
 'shoulder',
 'sliver',
 'acknowledge',
 '6-10pm',
 'qqlzk2jkzR',
 'hotspot',
 '7th',
 'record-',
 'FLL',
 'thS10LDY2a',
 'Honululu',
 'un-delay',
 '#media',
 'coffeemaker',
 'evaluate',
 'speechless',
 'interim',
 'bitcoin',
 '#Chairman',
 'turbulence',
 'role-model',
 '#lovejetblue',
 'UA6',
 'exact',
 'sob',
 'doesn',
 'efficient',
 'jx4s2t',
 'misinformation',
 'K7uBOTMr1r',
 'jetgreen',
 'soft',
 '#whereismybag',
 'here',
 'strong',
 'EU',
 'EVERYTIME',
 'unreLate',
 '@ti2469',
 'associate',
 'outdoor',
 'whether',
 'er',
 'Stacy',
 'swoop',
 'torture',
 'km',
 'advisory-now',
 '3231DTW',
 '443min',
 '@USChamber',
 'Yup',
 'assholes',
 'nozzle',
 'tsaprecheck',
 '645',
 'bar',
 '#BundleUp',
 '50',
 ' 330..',
 'asset',
 'diff',
 'unaccomidating',
 '@aircanada',
 'suitcase',
 'gal',
 'wanna',
 'EXTREMELY',
 '#2053',
 'PU',
 'jan',
 

<font size = 6 color="red"> See, how dictionary is build using only training data </font>

In [24]:
# how many words in vocabulary
num_unique_words = len(unique_word)
num_unique_words

11189

We will convert each **num_messages** messages into **num_unique_words** dimensional vector. Each entry in the vector counts number of times a word occurs in the text.  In machine earning we call activity like this **feature engineering**. 

First we need to assign each word a unique location/index in a **num_unique_words** dimensional vector.

In [25]:
#let' first  build a default dictionary to assign each word a unique location in the feature vector
from collections import defaultdict, Counter
word_to_index_dict = defaultdict(int)
for index , word in enumerate(unique_word):
    word_to_index_dict[word] = index
print (word_to_index_dict)    

defaultdict(<class 'int'>, {'bad': 0, 'tmrw': 1, 'mistreat': 2, 'CAE': 3, 'former': 4, 'nbr': 5, 'dream': 6, 'myself': 7, 'dunno': 8, '8273993': 9, 'Airserv': 10, 'functionally': 11, 'Gonna': 12, '#luvforSW': 13, 'shoulder': 14, 'sliver': 15, 'acknowledge': 16, '6-10pm': 17, 'qqlzk2jkzR': 18, 'hotspot': 19, '7th': 20, 'record-': 21, 'FLL': 22, 'thS10LDY2a': 23, 'Honululu': 24, 'un-delay': 25, '#media': 26, 'coffeemaker': 27, 'evaluate': 28, 'speechless': 29, 'interim': 30, 'bitcoin': 31, '#Chairman': 32, 'turbulence': 33, 'role-model': 34, '#lovejetblue': 35, 'UA6': 36, 'exact': 37, 'sob': 38, 'doesn': 39, 'efficient': 40, 'jx4s2t': 41, 'misinformation': 42, 'K7uBOTMr1r': 43, 'jetgreen': 44, 'soft': 45, '#whereismybag': 46, 'here': 47, 'strong': 48, 'EU': 49, 'EVERYTIME': 50, 'unreLate': 51, '@ti2469': 52, 'associate': 53, 'outdoor': 54, 'whether': 55, 'er': 56, 'Stacy': 57, 'swoop': 58, 'torture': 59, 'km': 60, 'advisory-now': 61, '3231DTW': 62, '443min': 63, '@USChamber': 64, 'Yup': 

In [26]:
# Let's create a reverse dictionary  for mapping index to word. It will help in debugging etc.
# See how we used dictionary comprehension
index_to_word_dict = { value:key  for key, value in word_to_index_dict.items()}

In [27]:
num_messages = training_messages.shape
num_messages

(9232,)

Let's create a numpy integer matrix of **num_messages X num_unique_words**, initialized with zero

In [28]:
# each row in training_X is our x_i
training_X = np.zeros((len(training_messages), len(unique_word)), dtype=int)
print(training_X.shape)
test_X = np.zeros((len(test_messages), len(unique_word)), dtype=int)

(9232, 11189)


In [29]:
def build_feature(text, word_to_index_dict):
    feature = np.zeros((len(word_to_index_dict),), dtype=int)
    word_freq =  Counter(text)
    # setting the word count in text_no row of text_features
    for word, freq in word_freq.items():
        if word in word_to_index_dict:
            index_of_word = word_to_index_dict[word]
            feature[index_of_word] = freq
    return feature        
for text_no, text in enumerate(training_messages):
    training_X[text_no] = build_feature(text, word_to_index_dict)
    
for text_no, text in enumerate(test_messages):
    test_X[text_no] = build_feature(text, word_to_index_dict)


## How we check if it above code is right ?
## Let's do some primitive checking on a text message

In [30]:
text_no =7
message_word_count = Counter(training_messages.iloc[text_no])
print(message_word_count)

# Let' check non zero location in text_features to see if count is set properly
print('##Encoding for text no {} in feature vector is ##'.format(text_no))
for i, count in enumerate(training_X[text_no]):
    if count >0:
        print(index_to_word_dict[i], count)


Counter({'.': 2, '@USAirways': 1, 'three': 1, 'hour': 1, 'on': 1, 'hold': 1, 'then': 1, 'disconnect': 1, 'Conf': 1, 'EBHSET': 1, 'need': 1, 'to': 1, 'be': 1, 'home': 1, 'Sunday': 1, 'for': 1, 'interview': 1})
##Encoding for text no 7 in feature vector is ##
to 1
for 1
three 1
interview 1
@USAirways 1
need 1
on 1
disconnect 1
Conf 1
be 1
EBHSET 1
Sunday 1
home 1
hour 1
. 2
then 1
hold 1


<font color= 'red' > Make sure Counter and encoding gives same results in above cell </font>

# We have successfully converted messages into a numpy matrix of feature vectors.

<img src="https://images.unsplash.com/photo-1532081274113-50aa6c277925?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1950&q=80" alt="Well done" width="500" height="400">


# Let's convert positive and negative label to 1 and 0  respectively

In [39]:
print (type(training_labels))
training_labels.head(10)# can check from head too

<class 'pandas.core.series.Series'>


9266     negative
9165     negative
10514    negative
4511     positive
9663     negative
14028    negative
11147    negative
10294    negative
5304     negative
7875     positive
Name: airline_sentiment, dtype: object

In [40]:
# This is our training_y value
training_y = (training_labels.values == 'positive').astype(int)
test_y = (test_labels.values == 'positive').astype(int)
training_y[9225:]

array([1, 1, 0, 1, 0, 0, 0])

# Training the model or estimating parameters $\theta$ of the model
Now we have vector feature representation $x_i$ of our text samples. 

Let review some theory and see what parameters we need to estimate for Naive bayes model.

We know that we classify a text $x_i$  to a class c= POSITIVE or c= NEGATIVE which has maximum value of $P(c|x_i).$ Using bayes rule we have $P(c|x_i) = \frac{P(x_i|c) P(c)}{P(x_i)} \propto P(x_i|c) P(c)$ as normalization doesn't depend on class label. 

In naive bayes assumption for modeling class conditional densities, we have $P(x_i|c) = \prod_j^D P(x_{ij}|c)$ assuming  $x_i \in \mathbb{R}^D$

**Note:$D$ is size of our vacabulary ($|V|$) build from text document corpus i.e D = |V|**

**what probability distribution we should choose for $P(x_{ij}|c)?$ **

Each value $x_{ij}$ is an integer values and there are total $D$ different unique values(word). This definetly suits a **$D$ side die** situation. 

**Infact, once we have learned $P(x_{ij}|c)?$ i.e probabilites of different sides for positive and negative die,**
** positive or negative text generation in bag of word model is nothing but rolling positive or negative die. Pick the word dictated by the side of die throw.**

Now we  know that we can put multinomial distribution for such situation. Hence
<font size = 6> 
$P(x_i|c) = \frac{n_i !}{\prod_j^D x_{ij|C}!} P(c) \prod^{D} P(w_j|c)^{x_{ij}} \propto P(c) \prod^{D} P(w_j|c)^{x_{ij}}$ 
</font>
as normalization doesn't depend on class label

We know that using MLE estimate we have
<font size = 8> 
$P(w_j|c) = \frac{\sum_{i=1}^N x_{ij}\mathbb{1}(y_i=c)}{\sum_{k=1}^{D} \sum_{i=1}^N x_{ik}\mathbb{1}(y_i=c)}.$ 
</font>
where $\mathbb{1}$ is indicator function.


- Hence the parameters are nothing  nothing but relative frequency of $w_j$ in documents of class c=NEGATIVE or c= POSITIVE
with respect to the total number of words in documents of that class.

- We can sum our numpy text_feature matrix along row or dim 0 to get total frequency of each feature for positive and negative class
- normalize total frequency of each feature with total frequency of all the features for each class.
- prior class  densities are estimated as $P(c) = \frac{N_c}{N}.$ Where $N_c$ are number of document/text in class k.

- So we have to learn **num_unique_words** parameters for each die
- and 2  class densities.


# Let's learn the parameters for c= positive(1) and c= negative(0)



In [33]:
# First estimate for positive

# summing up per feature count
print(training_X.shape)
training_X_positive = training_X[training_y ==1]
print(len(training_X_positive))
print(training_X_positive.shape)
#print(training_X_positive[:5])
per_feature_count =np.sum(training_X_positive, axis = 0)+1
print(per_feature_count.shape)
print(len(per_feature_count))
print(per_feature_count[:10])
print(np.sum(per_feature_count)+len(per_feature_count))
print(np.count_nonzero(per_feature_count))
parameters_w_positive = per_feature_count/(np.sum(per_feature_count))

parameters_w_positive[:5]

(9232, 11189)
1890
(1890, 11189)
(11189,)
11189
[13  2  1  1  2  1  4  3  1  1]
54812
11189


array([2.98007932e-04, 4.58473741e-05, 2.29236870e-05, 2.29236870e-05,
       4.58473741e-05])

# Let's estimate parameters for negative

# Q 3(.5 Point): Write code similarly to negative class to estimate following parameter parameters_w_negative for negative class.

In [34]:
# Write code here
training_X_negative = training_X[training_y ==0]
print(training_X_negative.shape)
per_feature_count = np.sum(training_X_negative, axis = 0)+1
print(per_feature_count.shape)
print(np.count_nonzero(per_feature_count))
parameters_w_negative = per_feature_count/(np.sum(per_feature_count)+len(per_feature_count))

print(parameters_w_negative)

NEGATIVE_CLASS, POSITIVE_CLASS = range(2)
print(type(NEGATIVE_CLASS))
print(NEGATIVE_CLASS)
print(POSITIVE_CLASS)

(7342, 11189)
(11189,)
11189
[6.75331692e-04 6.75331692e-05 1.03897183e-05 ... 1.55845775e-05
 1.03897183e-05 1.55845775e-05]
<class 'int'>
0
1


# Zero probability issue
As we can see some of the probability can be zero. It will create problem when we estimate probability of a new document in test set if that was not in training set. 

If any of the term in product is zero it will result in zero product. If any of the class don't have this term then probability of this document for any class will be zero. If we play log trick for comparing product of probability, we will be in trouble as log of zero is not defined.

**One way to handle this situtation to add a fake 1 count of the word in each class. This is called Laplace law of succession or add one smoothing**.

We estimate
<font size = 8> 
$P(w_j|c) = \frac{\sum_{i=1}^N x_{ij}\mathbb{1}(y_i=c) + 1}{\sum_{k=1}^{D} \sum_{i=1}^N x_{ik}\mathbb{1}(y_i=c) + |V|}.$ 
</font>
where $\mathbb{1}$ is indicator function and $|V|$ is size of our dictionary.

This can be done by adding a row of ones to training_X_positive and training_X_negative


# Once we learn have learned the model(i.e its parameters, probabilities of different words occuring in positive die and negative die), how good is our model ?
- Let take out our test data convert to count feature vector using same dictionaries
- Calculate the probability if test data belonging to Positive or negative. i.e if probability is >=.5 Positive otherwise negative
 or we can calculate the ratio
 <font size = 5>
 $\frac{P(x_{test}|c=positive)}{P(x_{test}|c=negative)} = \frac{ P(c=positive)  \prod^{D}_{j =1} P(w_j|c=positive)^{x_{test,j}}} { P(c= negative)\prod^{D}_{j=1} P(w_j|c=negative)^{x_{test, j}}}$ 
 </font>
 
 **Note:Generally such large product of probabilities, turns out to be zero because of computer representation limits of real numbers.**
 
 Another option is let take log on right hand side and after some manipulation one can show that if
 <font size = 5>
  $\sum_{j =1}^{D} (x_{test,j})log (P(w_j|c=positive)) +log(P(c= positive)) \ge log(P(c=negative))+ \sum_{j =1}^{D} (x_{test,j})log (P(w_j|c=negative))$
  
  </font>
  
 then it is positive otherwise negative
 
 
 
 
- Compare with test data label and let's report accuracy and confusion matrix

Let's do these steps

# Let's build a class for our multinomial naive bayes and estimate its new parameters based on  Laplace law of succession or add one smoothing.

You will use this same class on test challenge datasets.

# Q 4 (3 = .5 + 2(1+1)+ .5) Perform the following activity in the following multinomial_naive_bayes class 

- Implement \_calculate\_prior function
- Finish fit function
- Finish accuracy calculation

In [35]:
class multinomial_naive_bayes(object):
    
    NEGATIVE_CLASS, POSITIVE_CLASS = range(2)
    
    def __init__(self, class_prior=(.5, .5), fit_prior=True):
        
        self._class_prior= class_prior
        self._fit_prior = fit_prior
        
        
    def fit(self, training_X, training_y):
        '''
        Finishi this function. Store the parameters
        for positive class in self._parameters_w_positive
        for negative class in self._parameters_w_negative
        See how predict method relies on these paramters.
        '''
        #Write code here for estimating self._parameters_w_positive and self._parameters_w_negative
        training_X_positive = training_X[training_y==1]
        training_X_negative = training_X[training_y==0]
        pos_per_feature_count = np.sum(training_X_positive, axis = 0)+1
        neg_per_feature_count = np.sum(training_X_negative, axis = 0)+1
        #print(len(training_X_positive))
        #print(len(training_X_negative))
        self._parameters_w_positive = pos_per_feature_count/(np.sum(pos_per_feature_count)+len(pos_per_feature_count))
        self._parameters_w_negative = neg_per_feature_count/(np.sum(neg_per_feature_count)+len(neg_per_feature_count))
        if self._fit_prior:
            self._class_prior = self._calculate_prior(len(training_X_positive), len(training_X_negative))
            print(self._class_prior)
        #print('negative_params:' + str(self._parameters_w_negative))
        #print('positve+params:' + str(self._parameters_w_positive))
        #print(self._class_prior)
        return self    
            
    def _calculate_prior(self, n_positive, n_negative):    
        '''
        Finish this function to return prior of negative and positive class
        comment or remove the NotImplementedError 
        '''
        print("calculating priors")
        print("number of positive tweets:  " + str(n_positive))
        print("number of negative tweets:  " + str(n_negative))
        print("total number of training tweets:  " + str(len(training_X)))
        pos_prior = n_positive/len(training_X)
        neg_prior = n_negative/len(training_X)
        print("return a tuple: first element = negatives, second element = positives")
        return (neg_prior,pos_prior)
        #raise NotImplementedError("implement this!")
        
        
    def _calculate_score(self, parameters,test_text, class_prior):
            
            #print(parameters)
            #print(test_text)
            #print(class_prior)
            return np.sum(nnp.log(p.power(parameters,test_text))) + class_prior            
        
    def predict(self, test_X, test_y):
        '''
        predict the accuracy
        '''
        positive_score = np.zeros_like(test_y,dtype=float)
        negative_score = np.zeros_like(test_y,dtype=float)
        print ("shape of arrays to store positive and negative scores" + str(positive_score.shape) + ":" + str(negative_score.shape)) # just printing to make sure shape is right


        for idx, test_text in enumerate(test_X):# this will fetch row by row, encoded test messages
            positive_score[idx] = self._calculate_score(self._parameters_w_positive,
                                                       test_text,
                                                        np.log(self._class_prior[multinomial_naive_bayes.POSITIVE_CLASS]))
            negative_score[idx] = self._calculate_score(self._parameters_w_negative,
                                                        test_text, np.log(self._class_prior[multinomial_naive_bayes.NEGATIVE_CLASS]))
        print("first 20 scores from positive array:\n" + str(positive_score[:20]))
        print("first 20 scores from negative array:\n" + str(negative_score[:20]))
        
        ## Write code here
        #Look at the equation above  for deciding the class label and finish 
        # the code for accuracy calculation
        self.class_label = np.zeros_like(test_y,dtype=float)
      #  np.set_printoptions(threshold=np.inf)
        #self.class_label = positive_score - negative_score
        for idx in range(0,len(self.class_label)) :
             self.class_label[idx] = positive_score[idx] - negative_score[idx]
        #print(self.class_label[:5])   
             if self.class_label[idx] > 0 :
                self.class_label[idx] = 1 
             else:
                self.class_label[idx] = 0
        print("class label subtracts negatives scores from positive one and sets positive scores to 1:\n" + str(self.class_label[:20]))
        print("first 20 true lablels from test_y:\n" + str(test_y[:20]))
        self.accuracy_test = np.zeros_like(test_y,dtype=float)
        for idx in range(0,len(self.class_label)) :
              if self.class_label[idx] == test_y[idx] :
                    self.accuracy_test[idx] = 1
        print("if prediction matches label in test_y set it to 1, otherwise 0\n" + str(self.accuracy_test[:20]))  
        print("note: first non-matching one is 11th element")
        
        print("how many nonzero elements: (ones we got right)" + str(np.count_nonzero(self.accuracy_test)))
           
        print("hown many total elements in the test label:" + str(len(self.accuracy_test)))
        print("accuracy is total right/total in set")
        accuracy = np.count_nonzero(self.accuracy_test)/len(self.accuracy_test)
            
        return accuracy
        
    

In [36]:
clf = multinomial_naive_bayes()

In [37]:
clf.fit(training_X, training_y)


calculating priors
number of positive tweets:  1890
number of negative tweets:  7342
total number of training tweets:  9232
return a tuple: first element = negatives, second element = positives
(0.7952772963604853, 0.20472270363951472)


In [38]:
clf.predict(test_X, test_y)

shape of arrays to store positive and negative scores(2309,):(2309,)


NameError: name 'nnp' is not defined

<font color = 'BlueViolet' size = 6> Following code will show why python eco system shines.
We will use python library sklearn to build  multinomial Naive Bayes classifier </font>

# Q 5 (.5 = .25(training part) +.25 point(evaluation part)) Write code here to train MultinomialNB classifer  from sklearn and report accuracy in test set

Check if you get same accuracy number

In [41]:
from sklearn.naive_bayes import MultinomialNB
# Write the code here
clf = MultinomialNB()
clf.fit(training_X, training_y)
clf_pred = clf.predict(test_X)
print(clf_pred[4])
clf.score(test_X, test_y)


0


0.916847119965353